In [1]:
import pandas as pd
import numpy as np
from itertools import combinations, product

In [2]:
# read in manually updated csvs
drivers = pd.read_csv('f1 - drivers.csv')
constructors = pd.read_csv('f1 - constructors.csv')

In [3]:
# points per position: how many points does each race position score, on average?
ppp = drivers.sort_values('Season Points per Race',ascending=False)['Season Points per Race']

In [4]:
# expected finish order
efo = drivers.sort_values(['Implied Win Probability','Season Points per Race'],ascending=False)[['Driver','Season Points per Race','Cost','Constructor']]

In [5]:
# expected points (weighting is untested)
efo['Avg. Points per Position'] = list(ppp)
efo['Expected Points'] = 0.75*efo['Avg. Points per Position'] + 0.25*efo['Season Points per Race']
drivers = efo

In [6]:
# list every combination of 5 drivers 'dri'
dri = []
for driver_combo in combinations(drivers['Driver'].values,5):
    dri.append(driver_combo)

In [7]:
# list every combination of 2 constructors 'con'
con = []
for con_combo in combinations(constructors['Team'].values,2):
    con.append(con_combo)

In [8]:
# list every combination of those dri and con 'lineup'
lineup = []
for lineup_product in product(dri,con):
    lineup.append(lineup_product)

In [9]:
# pull the lineups into a dataframe
rows = []
for entry in lineup:
    d1 = entry[0][0]
    d2 = entry[0][1]
    d3 = entry[0][2]
    d4 = entry[0][3]
    d5 = entry[0][4]
    c1 = entry[1][0]
    c2 = entry[1][1]
    row = [d1,d2,d3,d4,d5,c1,c2]
    rows.append(row)
    
cols = ['D1','D2','D3','D4','D5','C1','C2']
df = pd.DataFrame(rows,columns=cols)

In [10]:
# map costs back onto dataframe
driver_map = drivers[['Driver','Cost']].set_index('Driver')['Cost'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Cost']].set_index('Team')['Cost'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_budget'
    df[new_col_str] = df[col].map(con_map)

df['total_budget'] = df[['D1_budget','D2_budget','D3_budget','D4_budget','D5_budget','C1_budget','C2_budget']].sum(axis=1)

In [11]:
# sum constructor expected points from its drivers
constructors['Expected Points'] = constructors['Team'].map(drivers.groupby('Constructor')['Expected Points'].sum())

In [12]:
# map win probabilities back onto dataframe
driver_map = drivers[['Driver','Expected Points']].set_index('Driver')['Expected Points'].to_dict()

for col in ['D1','D2','D3','D4','D5']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(driver_map)

con_map = constructors[['Team','Expected Points']].set_index('Team')['Expected Points'].to_dict()

for col in ['C1','C2']:
    new_col_str = col + '_xp'
    df[new_col_str] = df[col].map(con_map)

# the top driver you assign your 'DRS Boost' to gets their score doubled
df['D1_xp'] = df['D1_xp']*2

df['total_xp'] = df[['D1_xp','D2_xp','D3_xp','D4_xp','D5_xp','C1_xp','C2_xp']].sum(axis=1)

In [13]:
# reset index
df = df.reset_index()

In [14]:
# list the remaining lineups by combined expected points
drop = ['D1_xp', 'D2_xp', 'D3_xp', 'D4_xp','D5_xp', 'C1_xp', 'C2_xp','D1_budget','D2_budget', 'D3_budget', 'D4_budget', 'D5_budget', 'C1_budget','C2_budget']
df.drop(columns=drop,inplace=True)

In [15]:
# sort by winningest
df.sort_values(['total_xp','total_budget'],ascending=[False,True],inplace=True)

In [16]:
def best_available_lineups(df,team:list,budget:float,free_transfers:int):
    '''
    given a dataframe of upcoming race probs and your team info, spits out the best lineups to switch to
    
    df: pandas dataframe of all possible lineups, with corresponding total_budget and total_wp
    team_n: a list of the team's current entrants
    budget: the team's current cost cap
    free_transfers: the number of free transfers available
    
    '''
    df = df[df['total_budget'] <= budget] # filter by budget

    # subset to teams that share enough entrants with the current entry
    # to stay under the free transfer limit
    t1 = set(team)
    match_indices = [] # empty list to store match indices
    for possible_entry in df.values:
        t2 = set(possible_entry)
        if len(t1&t2) >= (7 - free_transfers):
            match_indices.append(possible_entry[0])
    df = df[df['index'].isin(match_indices)]
    return df

In [17]:
# team 1
team_1 = ['Max Verstappen','Yuki Tsunoda','Carlos Sainz','Fernando Alonso','Isack Hadjar','McLaren','Racing Bulls']
t1_new = best_available_lineups(df,team_1,budget=111.7,free_transfers=2)
t1_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
146748,146748,Max Verstappen,Oliver Bearman,Isack Hadjar,Carlos Sainz,Fernando Alonso,McLaren,Williams,110.3,152.450
144772,144772,Max Verstappen,Oliver Bearman,Yuki Tsunoda,Isack Hadjar,Fernando Alonso,McLaren,Haas,110.7,151.950
107600,107600,Max Verstappen,Kimi Antonelli,Isack Hadjar,Carlos Sainz,Fernando Alonso,McLaren,Aston Martin,110.5,151.950
145132,145132,Max Verstappen,Oliver Bearman,Yuki Tsunoda,Carlos Sainz,Fernando Alonso,McLaren,Haas,111.5,151.725
146752,146752,Max Verstappen,Oliver Bearman,Isack Hadjar,Carlos Sainz,Fernando Alonso,McLaren,Haas,105.8,150.400


In [18]:
# team 2
team_2 = ['Oscar Piastri','Nico Hulkenberg','Isack Hadjar','Fernando Alonso','Carlos Sainz','McLaren','Red Bull']
t2_new = best_available_lineups(df,team_2,budget=116.5,free_transfers=2)
t2_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
165062,165062,Max Verstappen,Isack Hadjar,Carlos Sainz,Fernando Alonso,Nico Hulkenberg,McLaren,Mercedes,116.5,171.75
391546,391546,George Russell,Oliver Bearman,Isack Hadjar,Carlos Sainz,Fernando Alonso,McLaren,Red Bull,115.9,166.60
473447,473447,Oscar Piastri,Oliver Bearman,Isack Hadjar,Carlos Sainz,Fernando Alonso,McLaren,Mercedes,115.4,166.50
391996,391996,George Russell,Oliver Bearman,Isack Hadjar,Fernando Alonso,Nico Hulkenberg,McLaren,Red Bull,115.6,164.20
473897,473897,Oscar Piastri,Oliver Bearman,Isack Hadjar,Fernando Alonso,Nico Hulkenberg,McLaren,Mercedes,115.1,164.10


In [19]:
# team 3
team_3 = ['George Russell','Carlos Sainz','Pierre Gasly','Fernando Alonso','Isack Hadjar','McLaren','Mercedes']
t3_new = best_available_lineups(df,team_3,budget=109.8,free_transfers=2)
t3_new.head()

,index,D1,D2,D3,D4,D5,C1,C2,total_budget,total_xp
409907,409907,George Russell,Isack Hadjar,Carlos Sainz,Fernando Alonso,Pierre Gasly,McLaren,Mercedes,109.6,162.925
411122,411122,George Russell,Isack Hadjar,Fernando Alonso,Liam Lawson,Pierre Gasly,McLaren,Mercedes,109.0,161.075
413642,413642,George Russell,Carlos Sainz,Fernando Alonso,Liam Lawson,Pierre Gasly,McLaren,Mercedes,109.8,160.850
410042,410042,George Russell,Isack Hadjar,Carlos Sainz,Fernando Alonso,Franco Colapinto,McLaren,Mercedes,108.8,160.600
411707,411707,George Russell,Isack Hadjar,Fernando Alonso,Pierre Gasly,Esteban Ocon,McLaren,Mercedes,109.8,160.575
